In [ ]:
from pathlib import PurePath
from palettable.colorbrewer.diverging import RdBu_5
from palettable.colorbrewer.sequential import Reds_5

from raster_compare.base import PixelClassifier, RasterFile
from raster_compare.plots import PlotBase

from plot_helpers import *

RED_BLUE = RdBu_5.mpl_colormap
REDS = Reds_5.mpl_colormap

ortho_image = plt.imread(
#     str(HOME_DIR / 'ERW_20180524_Agisoft_rgb_5m_ortho_2kx2k.tif')
    str(HOME_DIR / 'ERW_20180524_Agisoft_rgb_5m_ortho_2kx2k_tree.tif')
)

hillshade = RasterFile(HILLSHADE, band_number=1).band_values()


def style_area_axes(axes):
    set_axes_style(axes)
    axes.set_xticks([])
    axes.set_yticks([])

In [ ]:
casi_data = PixelClassifier(
#     HOME_DIR / 'ERW_20180524/CASI/CASI_2018_05_24_weekly_mosaic_classified_lidargrid_2kx2k_1m.tif',
    HOME_DIR / 'ERW_20180524/CASI/CASI_2018_05_24_weekly_mosaic_classified_lidargrid_2kx2k_1m_trees.tif',
    HOME_DIR / 'cloud_mask/cloud_mask_lidargrid_2kx2k_1m.vrt',
#     HOME_DIR / 'ERW_20180524/Lidar/ERW_20180524_Lidar_fr__lr_diff_2kx2k_1m.tif'
    HOME_DIR / 'ERW_20180524/Lidar/ERW_20180524_Lidar_fr__lr_diff_2kx2k_1m_tree.tif'
)

lidar_data = RasterFile(
#     HOME_DIR / 'ERW_snow_depth_lidar/ERW_snow_depth_lidar_lr_2kx2k_1m.tif', 
    HOME_DIR / 'ERW_snow_depth_lidar/ERW_snow_depth_lidar_lr_2kx2k_1m_tree.vrt', 
    band_number=1
)

band_values_lidar = lidar_data.band_values()
band_values_lidar_mask = band_values_lidar.mask.copy()

sfm_data = RasterFile(
#     HOME_DIR / 'ERW_snow_depth_sfm/ERW_snow_depth_sfm_lr_2kx2k_1m.tif', 
    HOME_DIR / 'ERW_snow_depth_sfm/ERW_snow_depth_sfm_lr_2kx2k_1m_tree.vrt', 
    band_number=1
)

band_values_sfm = sfm_data.band_values()
band_values_sfm_mask = band_values_sfm.mask.copy()

sd_diff_data = RasterFile(
#     HOME_DIR / 'ERW_snow_depth_diff/ERW_snow_depth_diff_2kx2k_1m.tif', 
    HOME_DIR / 'ERW_snow_depth_diff/ERW_snow_depth_diff_2kx2k_1m_tree.vrt', 
    band_number=1
)

band_values_sd_diff = sd_diff_data.band_values()
band_values_sd_diff_mask = band_values_sd_diff.mask.copy()

In [ ]:
band_values_lidar.mask = casi_data.snow_surfaces(band_values_lidar_mask)
band_values_sfm.mask = casi_data.snow_surfaces(band_values_sfm_mask)
band_values_sd_diff.mask = casi_data.snow_surfaces(band_values_sd_diff_mask)

In [ ]:
ticks = [-1.5, -1, -0.5, 0, 0.5, 1,  1.5]
tick_labels = ['< -1.5', '-1', '-0.5', '0', '0.5', '1', '> 1.5']

bins = np.arange(-1.50, 1.50 + 0.05, step=0.05)
bounds = dict(
    norm=colors.BoundaryNorm(
        boundaries=bins, ncolors=RED_BLUE.N,
    )
)

def add_hillshade(ax):
    ax.imshow(hillshade, cmap='gray', clim=(1, 255), alpha=0.5)

In [ ]:
figure = plt.figure(
    figsize=(12, 8),
    dpi=150,
    constrained_layout=False,
)
grid_spec = figure.add_gridspec(
    nrows=2, 
    ncols=3,
    wspace=0.0
)
ax1 = figure.add_subplot(grid_spec[0, 0])
add_hillshade(ax1)
ax1.imshow(
    band_values_lidar,
    cmap=RED_BLUE,
    **bounds
)
ax1.set_title('Lidar Snow Depth')

ax2 = figure.add_subplot(grid_spec[0, 1])
add_hillshade(ax2)
ax2.imshow(
    band_values_sfm,
    cmap=RED_BLUE,
    **bounds
)
ax2.set_title('SfM Snow Depth')

ax3 = figure.add_subplot(grid_spec[0, -1])
ax3.imshow(ortho_image)
ax3.add_artist(
        ScaleBar(5.0, location='lower left')
)
    
ax4 = figure.add_subplot(grid_spec[1, :-1])
data = prepare_data([
    band_values_lidar, band_values_sfm
])
hist_bins = np.arange(
    np.nanmin(data[0]),
    np.nanmax(data[0]),
    0.01
)
hist_lidar = ax4.hist(data[0], bins=hist_bins, color='dodgerblue', label='Lidar')
hist_sfm = ax4.hist(data[1], bins=hist_bins, color='brown', alpha=0.6, label='SfM')

vline_opts = dict(color='darkorange', linewidth=2)
label_y = max(hist_lidar[0].max(), hist_sfm[0].max())
# ax4.axvline(x=data[0].mean(), **vline_opts, ymax=0.82, label='Mean')
# ax4.annotate('Lidar', (data[0].mean() + .05, label_y * 0.78))
# ax4.axvline(x=data[1].mean(), **vline_opts, ymax=0.92)
# ax4.annotate('SfM', (data[1].mean() + .05, label_y * 0.9))
ax4.axvline(x=data[0].mean(), **vline_opts, ymax=0.95, label='Mean')
ax4.annotate('Lidar', (data[0].mean() + .05, label_y * 0.92))
ax4.axvline(x=data[1].mean(), **vline_opts, ymax=0.7)
ax4.annotate('SfM', (data[1].mean() - .25, label_y * 0.67))

ax4.axvline(x=0, color='black', linewidth=1)

ax4.ticklabel_format(style='sci', axis='y', scilimits=(4, 4))
ax4.yaxis.get_offset_text().set_visible(False)
ax4.set_xlabel("Snow Depth (m)")
ax4.set_ylabel("Count $(10^4)$")
ax4.set_xlim(-1.5, 3.5)
ax4.set_xticks([-1, -.5, 0, .5, 1, 2, 3])
ax4.legend()
ax4.set_title("Vegetated Area Distribution")
# ax4.set_title("High Elevation Distribution")

ax5 = figure.add_subplot(grid_spec[1, -1])
add_hillshade(ax5)
diff_plot = ax5.imshow(
    band_values_sd_diff,
    cmap=RED_BLUE,
    **bounds
)
ax5.set_title('Snow Depth Difference')

cbar = PlotBase.insert_colorbar(
    ax5, diff_plot,
    'Depth Difference (m)',
    ticks=ticks,
)
cbar.ax.set_yticklabels(tick_labels)

[style_area_axes(ax) for ax in [ax1, ax2, ax3, ax5]];

## Show positive snow depth values

In [ ]:
band_values_lidar.mask = (
    casi_data.snow_surfaces(np.ma.masked_less(band_values_lidar, 0).mask)
)
band_values_sfm.mask = (
    casi_data.snow_surfaces(np.ma.masked_less(band_values_sfm, 0).mask)
)
band_values_sd_diff.mask = (
    casi_data.snow_surfaces(np.ma.masked_less(band_values_sd_diff, 0).mask)
)

In [ ]:
figure = plt.figure(
    figsize=(12, 8),
    dpi=150,
    constrained_layout=False,
)
grid_spec = figure.add_gridspec(
    nrows=2, 
    ncols=3,
    wspace=0.0
)
ax1 = figure.add_subplot(grid_spec[0, 0])
add_hillshade(ax1)
ax1.imshow(
    band_values_lidar,
    cmap=RED_BLUE,
    **bounds
)
ax1.set_title('Lidar Snow Depth')

ax2 = figure.add_subplot(grid_spec[0, 1])
add_hillshade(ax2)
ax2.imshow(
    band_values_sfm,
    cmap=RED_BLUE,
    **bounds,
)
ax2.set_title('SfM Snow Depth')

ax3 = figure.add_subplot(grid_spec[0, -1])
ax3.imshow(ortho_image)
ax3.add_artist(
        ScaleBar(5.0, location='upper left')
)

ax4 = figure.add_subplot(grid_spec[1, :-1])
hist_bins = np.arange(
    np.nanmin(data[0]),
    np.nanmax(data[0]),
    0.01
)
ax4.hist(data[0], bins=hist_bins, color='dodgerblue', label='Lidar')
ax4.hist(data[1], bins=hist_bins, color='brown', alpha=0.6, label='SfM')

ax4.axvline(x=0, color='black', linewidth=1)
ax4.axvspan(-1.5, 0, alpha=0.5, color='grey')

ax4.ticklabel_format(style='sci', axis='y', scilimits=(4, 4))
ax4.yaxis.get_offset_text().set_visible(False)
ax4.set_xlabel("Snow Depth (m)")
ax4.set_ylabel("Count $(10^4)$")
ax4.set_xlim(-1.5, 3.5)
ax4.set_xticks([-1, -.5, 0, .5, 1, 2, 3])
ax4.legend()
ax4.set_title("High Elevation Distribution")

ax5 = figure.add_subplot(grid_spec[1, -1])
add_hillshade(ax5)
diff_plot = ax5.imshow(
    band_values_sd_diff,
    cmap=RED_BLUE,
    **bounds
)
ax5.set_title('Snow Depth Difference')

cbar = PlotBase.insert_colorbar(
    ax5, diff_plot,
    'Depth Difference (m)',
    ticks=ticks,
)
cbar.ax.set_yticklabels(tick_labels)

[style_area_axes(ax) for ax in [ax1, ax2, ax3, ax5]];

## Show areas of negative snow depth

In [ ]:
color_map = LinearSegmentedColormap.from_list(
    'snow_pixels',
    ['none', 'firebrick'],
    N=2
)

figure = plt.figure(
    figsize=(12, 8),
    dpi=150,
    constrained_layout=False,
)
grid_spec = figure.add_gridspec(
    nrows=2, 
    ncols=3,
    wspace=0.0
)
ax1 = figure.add_subplot(grid_spec[0, 0])
add_hillshade(ax1)
ax1.imshow(
    band_values_lidar < 0,
    cmap=color_map,
)
ax1.set_title('Lidar')

ax2 = figure.add_subplot(grid_spec[0, 1])
add_hillshade(ax2)
ax2.imshow(
    band_values_sfm < 0,
    cmap=color_map,
)
ax2.set_title('SfM')

ax3 = figure.add_subplot(grid_spec[0, -1])
ax3.imshow(ortho_image)
ax3.add_artist(
        ScaleBar(5.0, location='upper left')
)

ax4 = figure.add_subplot(grid_spec[1, :-1])
ax4.hist(data[0], bins=hist_bins, color='dodgerblue', label='Lidar')
ax4.hist(data[1], bins=hist_bins, color='brown', alpha=0.6, label='SfM')

ax4.axvline(x=0, color='black', linewidth=1)
ax4.axvspan(0, 3.5, alpha=0.5, color='grey')

ax4.ticklabel_format(style='sci', axis='y', scilimits=(4, 4))
ax4.yaxis.get_offset_text().set_visible(False)
ax4.set_xlabel("Snow Depth (m)")
ax4.set_ylabel("Count $(10^4)$")
ax4.set_xlim(-1.5, 3.5)
ax4.set_xticks([-1, -.5, 0, .5, 1, 2, 3])
ax4.legend()
ax4.set_title("High Elevation Distribution")

[style_area_axes(ax) for ax in [ax1, ax2, ax3]];

In [ ]:
color_map = LinearSegmentedColormap.from_list(
    'snow_pixels',
    ['none', 'mediumblue'],
    N=2
)

figure = plt.figure(
    figsize=(12, 8),
    dpi=150,
    constrained_layout=False,
)
grid_spec = figure.add_gridspec(
    nrows=2, 
    ncols=3,
    wspace=0.0
)
ax1 = figure.add_subplot(grid_spec[0, 0])
add_hillshade(ax1)
ax1.imshow(
    band_values_lidar > .75,
    cmap=color_map,
    alpha=0.9,
)
ax1.set_title('Lidar Snow Depth')

ax2 = figure.add_subplot(grid_spec[0, 1])
add_hillshade(ax2)
ax2.imshow(
    band_values_sfm > 0.75,
    cmap=color_map,
)
ax2.set_title('SfM Snow Depth')

ax3 = figure.add_subplot(grid_spec[0, -1])
ax3.imshow(ortho_image)
ax3.add_artist(
        ScaleBar(5.0, location='upper left')
)

ax4 = figure.add_subplot(grid_spec[1, :-1])
hist_bins = np.arange(
    np.nanmin(data[0]),
    np.nanmax(data[0]),
    0.01
)
ax4.hist(data[0], bins=hist_bins, color='dodgerblue', label='Lidar')
ax4.hist(data[1], bins=hist_bins, color='brown', alpha=0.6, label='SfM')
ax4.axvline(x=0.75, color='black', linewidth=1)
ax4.ticklabel_format(style='sci', axis='y', scilimits=(4, 4))
ax4.yaxis.get_offset_text().set_visible(False)
ax4.set_xlabel("Snow Depth (m)")
ax4.set_ylabel("Count $(10^4)$")
ax4.set_xlim(-1.5, 3.5)
ax4.set_xticks([-1, -.5, 0, .5, 1, 2, 3])
ax4.legend()

[style_area_axes(ax) for ax in [ax1, ax2, ax3]];